In [1]:
import numpy as np
import mmap
import warnings
import time
from concurrent.futures import ThreadPoolExecutor, Future
import concurrent.futures
warnings.filterwarnings('ignore')

In [2]:
length_numbers = 2**32
file_name = "Numbers.txt"
amount = 2*1024*1024*256
with open("Numbers.txt","wb") as f:
   array = np.random.randint(2, length_numbers, size=amount, dtype=np.dtype('uint32').newbyteorder('B')).byteswap()
   f.write(array.tobytes())
   f.close()

In [3]:
def get_time(func):
    start_time = time.clock()
    func()
    final_time = time.clock() - start_time
    print(str(final_time) + " seconds")



In [4]:
def println (Sum_number,Max_number,Min_number):
    print(f'Sum_number: \t{Sum_number}')
    print(f'Max_number: \t{Max_number}')
    print(f'Min_number: \t{Min_number}')


In [5]:
## Normal mode
def Normal_find():
    with open("Numbers.txt", "r+b") as f:
        data = f.read()
        array = np.frombuffer(data, dtype=np.dtype('uint32').newbyteorder('B'))
        print(len(array))
        print(array)
        Max_number = 0
        Min_number = 2**32
        Sum_number = 0

        for n in array:
            Sum_number += n
            if n > Max_number:
                Max_number = n
            if n < Min_number:
                Min_number = n
    f.close()
    println(Sum_number,Max_number,Min_number)
Normal_find()

536870912
[  17772286 3649248015  453614775 ... 1328523497  835213379 2898581224]
Sum_number: 	1152951861529982463
Max_number: 	4294967291
Min_number: 	9


In [6]:
## Mmap mode
def mmap_find():
    with open("Numbers.txt", "r+b") as f:
        with mmap.mmap(f.fileno(), length=length_numbers, offset=0, access=mmap.ACCESS_WRITE) as mm:
            array = np.frombuffer(mm, dtype=np.dtype('uint32').newbyteorder('B'))
            print(len(array))
            print(array)
            println(sum(array), array.max(), array.min())
            f.close()

mmap_find()

1073741824
[  17772286 3649248015  453614775 ...          0          0          0]
Sum_number: 	1152951861529982463
Max_number: 	4294967291
Min_number: 	0


In [7]:
#Multithreading and mapping mode
sum_number = 0
max_number = 0
min_number= 2**32

def search(array):
    global Sum_number
    global Max_number
    global Min_number

    for number in array:
        Sum_number += number
        if number > Max_number:
            Max_number = number
        if number < Min_number:
            Min_number = number

def mmp_thread_future_find():
    chunk = length_numbers // 12

    global Sum_number
    Sum_number = 0
    global Max_number
    Max_number = 0
    global Min_number
    Min_number = 2**32

    with open(file_name, 'r+b') as f:
        data = mmap.mmap(f.fileno(), length=length_numbers, offset=0, access=mmap.ACCESS_READ)
        array = np.frombuffer(data, dtype=np.dtype('uint32').newbyteorder('B'))
        print(len(array))
        print(array)
        with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
            arrays = [array[0: chunk], array[chunk: 2*chunk], array[2*chunk: 3*chunk], array[3*chunk:4*chunk],
                      array[4*chunk: 5*chunk], array[5*chunk: 6*chunk], array[6*chunk: 7*chunk],
                      array[7*chunk: 8*chunk],array[8*chunk: 9*chunk], array[9*chunk: 10*chunk],
                      array[10*chunk: 11*chunk],array[11*chunk: 12*chunk],array[12*chunk:]]
            future = {executor.submit(search, ar): ar for ar in arrays}

    println(Sum_number, Max_number, Min_number)

mmp_thread_future_find()

1073741824
[  17772286 3649248015  453614775 ...          0          0          0]
Sum_number: 	587656991947886997
Max_number: 	4294967291
Min_number: 	0


In [8]:
print("Normal mode running time")
get_time(Normal_find)

print("Mmap mode running time")
get_time(mmap_find)

print("Multithreading and mapping mode runtime")
get_time(mmp_thread_future_find)


Normal mode running time
1073741824
[  17772286 3649248015  453614775 ...          0          0          0]
Sum_number: 	1152951861529982463
Max_number: 	4294967291
Min_number: 	0
283.96853469999996 seconds
Mmap mode running time
1073741824
[  17772286 3649248015  453614775 ...          0          0          0]
Sum_number: 	1152951861529982463
Max_number: 	4294967291
Min_number: 	0
201.45101149999994 seconds
Multithreading and mapping mode runtime
1073741824
[  17772286 3649248015  453614775 ...          0          0          0]
Sum_number: 	618813832142090230
Max_number: 	4294967291
Min_number: 	0
293.7919187 seconds
